import time
Start the script run timer
script_start_time = time.time()

Top Site Selection Module

In [1]:
import argparse
import logging as log
import os
import pandas as pd
import geopandas as gpd

# local packages
import bc_combined_modelling.linking_utility as utils
import bc_combined_modelling.linking_wind as wind
import bc_combined_modelling.linking_vis as vis
import bc_combined_modelling.linking_solar as solar



In [2]:
# # Top Site Selection for the Resource Sites' Pool
# def main(
#     config_file_path:str,
#     resource_type: str):

In [3]:
# for notebook run
config_file_path='config/config_master.yml'
resource_type='wind'


log_path=f'workflow/log/{resource_type}_top_sites_module_log.txt'
utils.create_log(log_path)
utils.print_module_title(f'{resource_type} Top Sites Selection Module Initiated')

In [4]:
config=utils.load_config(config_file_path)
current_region=config['regional_info']['region_1']
_CRC_=current_region['code']
disaggregation_config=config['capacity_disaggregation'][f'{resource_type}']

log.info(f"Loading Configuration and Directories set-up...")
resource_max_capacity = disaggregation_config['max_capacity']  # GW
vis_dir=os.path.join(config['visualization']['linking'],resource_type)
linking_data:dict=config['processed_data']['linking']
result_files:dict=config['results']['linking']


log.info(f"Loading linking data files..")
cell_clusters=gpd.GeoDataFrame(pd.read_pickle(os.path.join(linking_data['root'],resource_type,linking_data[f'{resource_type}']['cell_clusters'])))
# dissolved_indices=utils.load_dict_datafile(os.path.join(wind_processed_data_directory,'dissolved_indices.json'))
if resource_type!='bess':
    dissolved_indices=pd.read_pickle(os.path.join(linking_data['root'],resource_type,linking_data[f'{resource_type}']['dissolved_indices']))
    province_grid_CF_ts_df=gpd.GeoDataFrame(pd.read_pickle(os.path.join(linking_data['root'],resource_type,linking_data[f'{resource_type}']['ERA5_CF_ts'])))
    scored_cells=gpd.GeoDataFrame(pd.read_pickle(os.path.join(os.path.join(linking_data['root'],resource_type,linking_data[f'{resource_type}']['scored_cells']))))

2024-09-18 12:33:51 - INFO - Loading Configuration and Directories set-up...
2024-09-18 12:33:51 - INFO - Loading linking data files..


In [5]:
# 1. Select the Top Sites 
selected_sites = utils.select_top_sites(cell_clusters, resource_max_capacity)

## Check if any sites were selected
if not selected_sites.empty:
    print(f"> {len(selected_sites)} Top Sites Selected. \n"
        f">> Total Capacity: {round(selected_sites['potential_capacity'].sum() / 1000, 2)} GW")
else:
    print("No sites selected.")

#just organizing the columns sequence and keeping the required columns only
selected_sites.loc[:,['Region', 'Cluster_No','Region_ID', 'potential_capacity',
     'nearest_station', 'nearest_station_distance_km','p_lcoe', 'geometry','capex', 'fom', 'vom']] #'Site_ID','CF_mean',

# # skipping geom data for nexus data prep
# columns_to_save = [col for col in selected_sites.columns if col != 'geometry']
# selected_sites_nexus_datafile = selected_sites[columns_to_save]

# selected_sites_nexus_datafile.to_csv((os.path.join(result_files['root'],result_files['clusters_topSites'][f'{resource_type}'])))

>>> Selecting TOP Sites to for 10 GW Capacity Investment in Province...
__________________________________________________
Selecting the Top Ranked Sites to invest in 10 GW resource in Province
__________________________________________________


!! Note: The Last cluster originally had 23.68 GW potential capacity.To fit the maximum capacity investment of 10 GW, it has been adjusted to 8.27 GW

> 6 Top Sites Selected. 
>> Total Capacity: 10.0 GW


,Region,Cluster_No,Region_ID,potential_capacity,nearest_station,nearest_station_distance_km,p_lcoe,geometry,capex,fom,vom
cluster_id,,,,,,,,,,,
Cowichan Valley_1,Cowichan Valley,1,27,121.136439,69_LCW_DSS,4.783322,593.643432,"MULTIPOINT (-124.10218 48.85952, -124.09968 48...",1.735275,0.041222,0
Mount Waddington_1,Mount Waddington,1,11,584.825537,138_CSS_GSS,0.135793,545.472146,"MULTIPOINT (-128.09967 50.76952, -128.09967 50...",1.735275,0.041222,0
Fraser Valley_1,Fraser Valley,1,21,68.441507,69_KEN_JCT,5.242786,482.779712,"MULTIPOINT (-122.13467 49.12702, -122.13218 49...",1.735275,0.041222,0
East Kootenay_1,East Kootenay,1,16,931.626793,69_CMO_DSS,1.565074,473.937572,"MULTIPOINT (-116.56218 50.81202, -116.55968 50...",1.735275,0.041222,0
Sunshine Coast_1,Sunshine Coast,1,19,22.905823,138_SCG_GSS,8.127362,429.462058,"MULTIPOINT (-123.67718 49.71952, -123.67467 49...",1.735275,0.041222,0
Peace River_1,Peace River,1,3,8271.063902,138_E15_JCT,0.15327,418.397214,"MULTIPOINT (-126.99968 56.96202, -126.99718 56...",1.735275,0.041222,0


In [6]:
# 2. Create/Load the Representative Timeseries for the Selected Sites
if resource_type=="bess":
    log.info(f"Time-slice creation and visuals not required for BESS.")
else:
    if resource_type=="wind":
        CF_ts_clusters_df,within_cluster_cells_ts_df= wind.create_timeseries_for_Cluster(selected_sites,dissolved_indices,scored_cells,province_grid_CF_ts_df)
        # CF_ts_clusters_df -> Clusters' representative timeseries
        # within_cluster_cells_ts_df -> individual GWA cell's timeseries
    elif resource_type=="solar": #kept it dedicated to solar as future expansion version will include other resources ~
        CF_ts_clusters_df=solar.create_timeseries_for_Cluster(selected_sites,dissolved_indices,province_grid_CF_ts_df)
        # within_cluster_cells_ts_df=province_grid_CF_ts_df
    
    
    ### Save Sites' Timeseries data file Locally
    # CF_ts_clusters_df_save_to=os.path.join(os.path.join(os.path.join(linking_data['root'],linking_data[f'{resource_type}']['cell_cluster_ts'])))
    # CF_ts_clusters_df.to_pickle(CF_ts_clusters_df_save_to)
    # log.info (f"Timeseries for {resource_type} - ALL sites created and saved locally at - {CF_ts_clusters_df_save_to}.")
    
# 3. Prepare top sites (clusters) representative timeseries

    CF_ts_df_Top_sites = CF_ts_clusters_df[selected_sites.index]
    CF_ts_df_Top_sites=utils.fix_df_ts_index(CF_ts_df_Top_sites,config['cutout']['snapshots_tz_BC'],snapshot_serial=0)

    ## Save File Locally
    result_files:dict=config['results']['linking']
    log.info (f"Plotting timeseries for {len(CF_ts_clusters_df)} top sites for {resource_type}...")
    CF_ts_df_Top_sites.to_pickle(os.path.join(result_files['root'],result_files['clusters_CFts_topSites'][f'{resource_type}']))
    log.info (f"Timeseries for {resource_type} top sites created and saved locally.")

# 3. Visualization of the Site - Timeseries
    
    plots_save_to=os.path.join(vis_dir,'Site_timeseries')
    # resampling_span:str=result_files['visual_resampling'][f'{resource_type}']
    # vis.create_timeseries_plots(selected_sites, within_cluster_cells_ts_df,resource_max_capacity,dissolved_indices,resampling_span,'blue','skyblue',plots_save_to) 
    # log.info(f"Static plots for {resource_type} site's timeseries created and saved locally at : '{plots_save_to}'") 
    
    vis.create_timeseries_interactive_plots(CF_ts_df_Top_sites,plots_save_to)
    log.info(f"Interactive plots for {resource_type} site's timeseries created and saved locally at : '{plots_save_to}'")
    
    vis.create_sites_ts_plots_all_sites(resource_type,CF_ts_df_Top_sites,plots_save_to)
    vis.create_sites_ts_plots_all_sites(resource_type,CF_ts_df_Top_sites,f'results/linking')


selected_sites['CF_mean']=selected_sites.index.map(CF_ts_df_Top_sites.mean())

## Save Site's Spatial, Technical data file Locally
selected_sites.to_pickle(os.path.join(result_files['root'],result_files['clusters_topSites'][f'{resource_type}']))


log.info (f"Top Sites Selection for {resource_type} - Execution Completed !")



2024-09-18 12:33:52 - INFO - Creating Timeseries for Cowichan Valley Region
2024-09-18 12:33:52 - INFO -  >> Creating Timeseries for cluster 1 / 1
2024-09-18 12:33:52 - INFO - Creating Timeseries for Mount Waddington Region
2024-09-18 12:33:52 - INFO -  >> Creating Timeseries for cluster 1 / 1
2024-09-18 12:33:54 - INFO - Creating Timeseries for Fraser Valley Region
2024-09-18 12:33:54 - INFO -  >> Creating Timeseries for cluster 1 / 1
2024-09-18 12:33:55 - INFO - Creating Timeseries for East Kootenay Region
2024-09-18 12:33:55 - INFO -  >> Creating Timeseries for cluster 1 / 1
2024-09-18 12:34:01 - INFO - Creating Timeseries for Sunshine Coast Region
2024-09-18 12:34:01 - INFO -  >> Creating Timeseries for cluster 1 / 1
2024-09-18 12:34:01 - INFO - Creating Timeseries for Peace River Region
2024-09-18 12:34:01 - INFO -  >> Creating Timeseries for cluster 1 / 1
2024-09-18 12:34:38 - INFO - Plotting timeseries for 8760 top sites for wind...
2024-09-18 12:34:38 - INFO - Timeseries for wi

# skip this part for notebook run

In [7]:
# End the script run  timer
script_end_time = time.time()

# Calculate runtime in seconds
runtime = round((script_end_time - script_start_time),2)

return log.info (f"Script runtime: {runtime} seconds")
if __name__ == "__main__":      

    # Set up argument parsing
    parser = argparse.ArgumentParser(description='Run data preparation script')
    parser.add_argument('config', type=str, help=f"Path to the configuration file 'config_master.yml'. Ideally in this directory: 'config/config_master.yml' ")
    parser.add_argument('resource_type', choices=['wind', 'solar','bess'], help="Type of resource: 'wind' or 'solar' or 'bess' ")
    # Parse the arguments
    
    #----------------Main code to be used ----------------
    args = parser.parse_args()
    main(args.config,args.resource_type)
    
    #----------------------- for notebook run/Debugging------------------------------------
    # config_file_path='config/config_master.yml'
    # main(config_file_path,resource_type='wind')

NameError: name 'time' is not defined